In [10]:
import os


In [11]:

GM_datapath = '/home/ec2-user/SageMaker/data/The_Globe_and_Mail_with_DP_filter_by_article_type/'
TS_datapath = '/home/ec2-user/SageMaker/data/Toronto_Star_Publication_with_query/'
GloVe_datapath ='/home/ec2-user/SageMaker/mariano/notebooks/03. High Recall Retrieval System/vectors/'
GM_filenames = os.listdir(GM_datapath)
print(f"Number of Globe and Mail articles: {len(GM_filenames):,}" )
GM_filenames.sort()
print(GM_filenames[:5])
print(GM_filenames[-5:])

# Number of Globe and Mail articles: 151,004
# ['1041435397.xml', '1124924513.xml', '1124924549.xml', '1124928180.xml', '1124937415.xml']
# ['2610545245.xml', '2611600803.xml', '2611601132.xml', '2611601516.xml', '2611601526.xml']

print()
TS_filenames = os.listdir(TS_datapath)
print(f"Number of Toronto Star articles: {len(TS_filenames):,}" )
TS_filenames.sort()
print(TS_filenames[:5])
print(TS_filenames[-5:])
# Number of Toronto Star articles: 41,429
# ['1000282449.xml', '1000282615.xml', '1000282859.xml', '1000282969.xml', '1000345426.xml']
# ['994013461.xml', '994013464.xml', '994013511.xml', '994014287.xml', '994377993.xml']

# There are no two articles with same ID (and filename)
assert len(TS_filenames)+len(GM_filenames) == len(set(TS_filenames).union(set(GM_filenames)))

Number of Globe and Mail articles: 151,004
['1041435397.xml', '1124924513.xml', '1124924549.xml', '1124928180.xml', '1124937415.xml']
['2610545245.xml', '2611600803.xml', '2611601132.xml', '2611601516.xml', '2611601526.xml']

Number of Toronto Star articles: 41,429
['1000282449.xml', '1000282615.xml', '1000282859.xml', '1000282969.xml', '1000345426.xml']
['994013461.xml', '994013464.xml', '994013511.xml', '994014287.xml', '994377993.xml']


In [12]:
import spacy
import numpy as np
nlp = spacy.load('en_core_web_lg', disable=['ner','parser','lemmatizer','tagger', 'textcat'])


 ### Globe and Mail

In [13]:
%%time
from lxml import etree
from bs4 import BeautifulSoup
import pickle

# We define a function to get the text content that we need from the XML articles available in our dataset
def getxmlcontent(root):
    if root.find('.//HiddenText') is not None:
        return(root.find('.//HiddenText').text)
    
    elif root.find('.//Text') is not None:
        return(root.find('.//Text').text)
    
    else:
        return None
    

# The Globe and Mail
count=0
for filename in GM_filenames:
    tree = etree.parse(GM_datapath + filename)
    root = tree.getroot()
    
    if getxmlcontent(root) is not None:
        soup = BeautifulSoup(getxmlcontent(root))
        text = soup.get_text()
    else:
        text = 'Error in processing document'                    

    vector_filename = filename[:-4]+ ".vector"    
    if not os.path.isfile(GloVe_datapath+vector_filename):
        pickle.dump(nlp(text).vector,open(GloVe_datapath+vector_filename, "wb"))
        count+=1
print(f'Number of GM files proccesed: {count}')
print(f'Number of GM files skipped: {len(GM_filenames)-count}')

# The Globe and Mail
count=0
for filename in TS_filenames:
    tree = etree.parse(TS_datapath + filename)
    root = tree.getroot()
    
    vector_filename = filename[:-4]+ ".vector"    
    if not os.path.isfile(GloVe_datapath+vector_filename):
        # Retrieve text
        if getxmlcontent(root) is not None:
            soup = BeautifulSoup(getxmlcontent(root))
            text = soup.get_text()
        else:
            text = 'Error in processing document'  
        # Process text and store vector
        pickle.dump(nlp(text).vector,open(GloVe_datapath+vector_filename, "wb"))     
        count+=1             


        
print(f'Number of TS files proccesed: {count}')
print(f'Number of TS files skipped: {len(TS_filenames)-count}')




Number of GM files proccesed: 0
Number of GM files skipped: 151004
Number of TS files proccesed: 41429
Number of TS files skipped: 109575
CPU times: user 35min 27s, sys: 13.1 s, total: 35min 40s
Wall time: 39min 52s


In [18]:
import os
import pickle

vectors_datapath = '/home/ec2-user/SageMaker/mariano/notebooks/03. High Recall Retrieval System/vectors/'
vector_filenames = [filename for filename in os.listdir(vectors_datapath) if filename.endswith('vector')]
print(f'Cantidad de vectores recuperados: {len(vector_filenames):,}')

id2vector = {}

for vector_filename in vector_filenames:
    id_ = int(vector_filename[:-7])
    id2vector[id_] = pickle.load(open(vectors_datapath+vector_filename, 'rb'))
    
dict_path = vectors_datapath+'id2vec.p'
pickle.dump(id2vector, open(dict_path, "wb"))


Cantidad de vectores recuperados: 192,433


In [19]:
len(pickle.load(open(dict_path,'rb')))

192433